In [2]:
import skimage.io as io
import time
import numpy as np
import cv2
import os

from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA

from Features.preprocessing import *
from Features.LocalBinaryPattern import *
from Features.ChainCodesFeatures import *
from Features.PolygonFeatures import * 
from Features.WindowFeatures import *
from Features.windows import *

In [3]:
def Feature_Extractor(img):
    img = cutHandWriting(img)
    
    img = cv2.resize(img,(img.shape[0]//4,img.shape[1]//4))

    
    #lbp = LocalBinaryPatterns(24,8)
    #lbp_features = lbp.describe(img)
    #st = time.time()
    edged = cv2.Canny(img, 30, 200)
    contours, hierarchy = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    #en = time.time()
#     print("Contours : ",(en-st)*1000,"ms")
    
    #st = time.time()
    #windows, clusters, clusteredWindows = getClusteredWindows(img,13,10)
    #en = time.time()
#     print("Kesho : ",(en-st)*1000,"ms")
    
    #st = time.time()
    F5 = Chain_Codes_Feature_Extractor(img,contours,None)
    #en = time.time()
#     print("Ramzy : ",(en-st)*1000,"ms")
    
    #st = time.time()
    #f10, f11, f12, f13, f14 = PolygonFeatures(contours)
    #en = time.time()
#     print("Ibrahim Lefta : ",(en-st)*1000,"ms")
    
    #st = time.time()
    #Windows = list()
    
    #for i in range(clusteredWindows.shape[0]):
        #print(clusteredWindows[i])
        #Windows  += list(WindowFeatures(clusteredWindows[i].astype(np.uint8)))
    #en = time.time()
#     print("October : ",(en-st)*1000,"ms")
    return list(F5)
    #return F5
    Features = list(F1)+list(F2)+list(F3)+list(F4)+list(F5)+list(F7)+list(F8)+list(F9)+list(f10)+list(f11)+list(f12)+list(f13)+list(f14)+list(lbp_features)+Windows

In [3]:
correct = []
f = open("correct.txt", "r")
for x in f:
    correct.append(int(x)) 

In [4]:
test_cases = os.listdir("TestData")
test_cases.sort()

In [5]:
i = 0 
acc = 0
for case in test_cases:
    writer1_1  = cv2.imread('TestData/'+case+"/1/1.png", cv2.IMREAD_GRAYSCALE)
    writer1_2  = cv2.imread('TestData/'+case+"/1/2.png", cv2.IMREAD_GRAYSCALE)
    writer2_1  = cv2.imread('TestData/'+case+"/2/1.png", cv2.IMREAD_GRAYSCALE)
    writer2_2  = cv2.imread('TestData/'+case+"/2/2.png", cv2.IMREAD_GRAYSCALE)
    writer3_1  = cv2.imread('TestData/'+case+"/3/1.png", cv2.IMREAD_GRAYSCALE)
    writer3_2  = cv2.imread('TestData/'+case+"/3/2.png", cv2.IMREAD_GRAYSCALE)
    test_img =  cv2.imread('TestData/'+case+"/test.png", cv2.IMREAD_GRAYSCALE)

    st = time.time()
    '''
    f1_1_5 , f1_1_11 , f1_1_13  = Feature_Extractor(writer1_1)
    f1_2_5 , f1_2_11 , f1_2_13  = Feature_Extractor(writer1_2)
    f2_1_5 , f2_1_11, f2_1_13  = Feature_Extractor(writer2_1)
    f2_2_5, f2_2_11 , f2_2_13 = Feature_Extractor(writer2_2)
    f3_1_5 , f3_1_11 , f3_1_13  = Feature_Extractor(writer3_1)
    f3_2_5 , f3_2_11 , f3_2_13  = Feature_Extractor(writer3_2)
    pred_f5 , pred_f11 , pred_f13  = Feature_Extractor(test_img)
    '''
    
    f1_1  = Feature_Extractor(writer1_1)
    f1_2  = Feature_Extractor(writer1_2)
    f2_1  = Feature_Extractor(writer2_1)
    f2_2 = Feature_Extractor(writer2_2)
    f3_1  = Feature_Extractor(writer3_1)
    f3_2  = Feature_Extractor(writer3_2)
    pred  = Feature_Extractor(test_img)
    
    features = np.array([f1_1,f1_2,f2_1,f2_2,f3_1,f3_2])
    #features_f5 = np.array([f1_1_5,f1_2_5,f2_1_5,f2_2_5,f3_1_5,f3_2_5])
    #features_f11 = np.array([f1_1_11,f1_2_11,f2_1_11,f2_2_11,f3_1_11,f3_2_11])
  # features_f11_f13 = np.array([list(f1_1_11)+list(f1_1_13),list(f1_2_11)+list(f1_2_13),list(f2_1_11)+list(f2_1_13),list(f2_2_11)+list(f2_2_13),list(f3_1_11)+list(f3_1_13),list(f3_2_11)+list(f3_2_13)])
    
    
    classifier = SVC()
    classifier.fit(features, [1,1,2,2,3,3])
    result = classifier.predict([pred])
    
    '''classifier_f11 = SVC()
    classifier_f11.fit(features_f11, [1,1,2,2,3,3])
    result_f11 = classifier_f11.predict([pred_f11])
    
    classifier_f11_f13 = SVC()
    classifier_f11_f13.fit(features_f11_f13, [1,1,2,2,3,3])
    result_f11_f13 = classifier_f11_f13.predict([list(pred_f11)+list(pred_f13)])
    result = np.array([result_f5[0],result_f11[0],result_f11_f13[0]])
    vote_1 = len(result[result==1])
    vote_2 = len(result[result==2])
    vote_3 = len(result[result==3])
    if vote_1 >= 2 :
        result = 1
    elif vote_2 >= 2 :
        result = 2
    elif vote_3 >=2 :
        result = 3
    else:
        result = result_f5
   '''
    
    
    en = time.time()
    print("Time Taken : ",(en-st))
    print(result,correct[i])
    if(result==correct[i]):
        acc+=1
    i+=1
    print(i,acc, "Accuracy : ",acc*100/i,"%")

Time Taken :  4.230611085891724
[3] 3
1 1 Accuracy :  100.0 %
Time Taken :  5.369417667388916
[3] 3
2 2 Accuracy :  100.0 %
Time Taken :  5.338851690292358
[3] 3
3 3 Accuracy :  100.0 %
Time Taken :  5.6315929889678955
[3] 3
4 4 Accuracy :  100.0 %
Time Taken :  5.497735977172852
[3] 3
5 5 Accuracy :  100.0 %
Time Taken :  4.467421531677246
[3] 3
6 6 Accuracy :  100.0 %
Time Taken :  4.700604677200317
[3] 3
7 7 Accuracy :  100.0 %
Time Taken :  5.261789083480835
[3] 3
8 8 Accuracy :  100.0 %
Time Taken :  5.252420663833618
[3] 3
9 9 Accuracy :  100.0 %
Time Taken :  5.185472249984741
[3] 3
10 10 Accuracy :  100.0 %
Time Taken :  5.245802164077759
[3] 3
11 11 Accuracy :  100.0 %
Time Taken :  5.007981777191162
[3] 3
12 12 Accuracy :  100.0 %
Time Taken :  4.8562116622924805
[3] 3
13 13 Accuracy :  100.0 %
Time Taken :  5.824315547943115
[3] 3
14 14 Accuracy :  100.0 %
Time Taken :  6.2858452796936035
[3] 3
15 15 Accuracy :  100.0 %
Time Taken :  4.742481470108032
[3] 3
16 16 Accuracy : 

# Results 
### SVM on F1 Feature : 95.7%  (68/71)
### SVM on F5 Feature : 97.2%  (69/71)
### SVM on F4 Feature : 97.2%  (69/71) With Resize /4  , 98.6% (70/71)  With Resize /2 
### SVM on F11 Feature : 94.3% (67/71)
### SVM on F11+F12+F13+F14   : 83.1% (59/71)
### SVM on  F11 + F13 : 93% (66/71)
###  3 SVM Classifiers   (F11+F13) , F11 , F5 : 95.7%  (68/71)

# Random Test


In [4]:
with open("forms.txt") as h:
    lines = h.readlines()[16:]

In [5]:
freq = np.zeros(680, dtype=int)
formWriter = {}
for i in lines:
    line = i.split(' ')
    key = line[1]
    if key in formWriter.keys():
        formWriter[key].append(line[0])
    else:
        formWriter[key] = [line[0]]

In [6]:
freq = [x for x, l in formWriter.items() if len(l)>2]

In [7]:
acc = 0
# total = 0
for itr in range(500):
    keys = set()
    while len(keys) !=3: #To avoid test image same as train images
            keys.add(np.random.randint(len(freq), size=1)[0])
    keys = list(keys)
    keys = np.random.randint(len(freq), size=3)
    train = []
    test = []
    for k in keys:
        forms = formWriter[freq[k]]
#         forms = [freq[k]+'-'+'{:03d}'.format(s)+'.jpg' for s in range(len(forms))]
        #idxes = np.random.randint(len(forms), size=3)
        idxes = set()
        while len(idxes) !=3: #To avoid test image same as train images
            idxes.add(np.random.randint(len(forms), size=1)[0])
        idxes = list(idxes)
        train.append(forms[idxes[0]])
        train.append(forms[idxes[1]])
        test.append(forms[idxes[2]])
    testWriter = np.random.randint(len(test))
    testImg = test[testWriter]
    
    trainFeatures = []
    Y = []
    for i in range(len(train)):
        img = cv2.imread('forms/'+train[i]+'.png', cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue
       
        f = Feature_Extractor(img)
        trainFeatures.append(f)
        Y.append(i//2)
    trainFeatures =  np.array(trainFeatures)
    classifier = SVC()
   
    classifier.fit(trainFeatures, Y)
    
    img = cv2.imread("forms/"+testImg+'.png', cv2.IMREAD_GRAYSCALE)
    if img is None:
        continue

    testFeatures = Feature_Extractor(img)
    result = classifier.predict([testFeatures])
    if result[0] == testWriter:
        acc += 1
#     total +=1
#     print(len(testFeatures))
#     print(total,acc, "Accuracy : ",acc*100/total,"%")
#     print(result, testWriter)
    print("Iteration: {} | Correct Predictions: {} | Prediction: {} | Truth: {} | Accuracy: {}%"\
              .format(itr+1, acc, result, testWriter, acc*100/(itr+1)))

Iteration: 1 | Correct Predictions: 1 | Prediction: [0] | Truth: 0 | Accuracy: 100.0%
Iteration: 2 | Correct Predictions: 2 | Prediction: [1] | Truth: 1 | Accuracy: 100.0%


KeyboardInterrupt: 

# Results 
### SVM :  Normal Windows , No Ali part  : 86%    (86/100)     , 83%
### KNN :  Normal Windows , No Ali part  : 74%    (74/100)
### SVM :  Normal Windows , No Ali part no local features (84/100)
### SVM :  Normal Windwos , No Ali part , Pca  :64% (7/11)
### SVM : F1  90% (449/500)
### SVM : F5  97.2% (484/498)
### SVM : F11 90.6% (453/500)
### SVM : F4  95.2%  (474/498)
### KNN : F5 93.4%  (467/500)